In [89]:
import numpy as np
from Bio import SeqIO
from collections import defaultdict
from copy import deepcopy
import networkx as nx 
import matplotlib.pyplot as plt 
import sys
from pyvis.network import Network

sys.setrecursionlimit(10000000)

In [16]:


np.random.seed(0)

nucleotides = ['A', 'T', 'G', 'C']

sequence_string = ''.join(np.random.choice(nucleotides, 1000).tolist())
reads = ''

for i in range(1, 1001):
    read_len = 150
    read_start = np.random.default_rng().integers(0, 1000 - read_len)
    read_str = sequence_string[read_start:read_start+read_len]
    reads += f'>READ_SEQ_{i}\n{read_str}\n'

with open(f"reads.fasta", "w") as file:
    file.write(reads)

with open("sequense.fasta", "w") as file:
    file.write('>MAIN_SEQ_0\n')
    file.write(sequence_string)

## Построение графа Де Брюина
По заданному набору ридов в формате FASTQ и параметру k, который соответствует длине k-меров, построить граф Де Брюина, некоторый путь в котором соответствовал бы возможной подстроке в исходном геноме. Не забывайте про запоминание покрытия каждого k-мера, а так же про сами подстроки, которые соответствуют каждому ребру. В остальном граф полностью соответствует тому, что был описан в лекции.

In [3]:
seq_name, seq_string = [], []
with open ("/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_2/reads.fasta",'r') as fa:
    for seq_record  in SeqIO.parse(fa,'fasta'):
        seq_name.append(str(seq_record.name))
        seq_string.append(str(seq_record.seq))

true_seq_name, true_seq_string = [], []
with open ("/home/pk/Desktop/BOTAY/BioinformaticsCourse2024PK/homework/2_2/sequense.fasta",'r') as fa:
    for seq_record  in SeqIO.parse(fa,'fasta'):
        true_seq_name.append(str(seq_record.name))
        true_seq_string.append(str(seq_record.seq))
true_seq = true_seq_string[0]

In [130]:
class Node():
    def __init__(self, string):
        self.string = string
        self.in_degree = 0
        self.out_degree = 0

class Edge():
    def __init__(self, prev_node, next_node, string, coverage=1):
        self.prev_node = prev_node
        self.next_node = next_node
        self.string = string
        self.coverage = coverage

class DBGraph():
    def __init__(self, reads, k):
        self.reads = reads
        self.k = k
        self.used_edges = set()
        self.used_nodes = set()
        self.nodes = {}
        self.edges = {}
        self.connections = defaultdict(list)
        self.kmers = []
        self.compacted = False
        self.deleted_tails = False
        self.string = None

    def gen_kmers(self):
        for read in self.reads:
            self.kmers.extend([read[i:self.k+i] for i in range(0, len(read)-self.k+1)])

    def create_graph(self):
        for kmer in self.kmers:
            prev_node_str = kmer[:-1]
            next_node_str = kmer[1:]
            if prev_node_str not in self.used_nodes:
                self.nodes[prev_node_str] = Node(prev_node_str)
                self.used_nodes.add(prev_node_str)
            if next_node_str not in self.used_nodes:
                self.nodes[next_node_str] = Node(next_node_str)
                self.used_nodes.add(next_node_str)
            if kmer in self.used_edges:
                self.edges[kmer].coverage += 1
            else:
                self.used_edges.add(kmer)
                self.edges[kmer] = Edge(prev_node_str, next_node_str, kmer)
                self.connections[prev_node_str].append(self.edges[kmer])
                self.nodes[prev_node_str].out_degree += 1
                self.nodes[next_node_str].in_degree += 1

    def compact(self):
        # if self.compacted:
        #     print('This graph has already been compacted')
        #     return
        to_delete = set()
        self.compacted = True
        self.connections_temp = deepcopy(self.connections)
        for node in self.nodes:
            # print(node)
            # print(self.nodes[node].in_degree)
            # print(self.nodes[node].out_degree)
            if self.nodes[node].out_degree >= 1:
                for new_edge_idx in range(len(self.connections[node])):
                    new_edge = self.connections[node][new_edge_idx]
                    next_edge = new_edge
                    # print(next_edge.next_node)
                    if self.nodes[new_edge.next_node].out_degree == 1 and self.nodes[new_edge.next_node].in_degree == 1:
                        while True:
                            prev_edge = new_edge
                            next_edge = self.connections[new_edge.next_node][0]
                            new_edge_string = prev_edge.string + next_edge.string[-1]
                            new_edge_coverage = int(len(prev_edge.string)/len(new_edge_string)*prev_edge.coverage + len(next_edge.string)/len(new_edge_string)*next_edge.coverage)
                            # self.nodes[prev_edge.next_node].is_compacted = True
                            self.nodes[prev_edge.next_node].in_degree -= 1
                            self.nodes[prev_edge.next_node].out_degree -= 1
                            del self.connections[prev_edge.next_node]
                            new_edge = Edge(node, next_edge.next_node, new_edge_string, new_edge_coverage)
                            if self.nodes[new_edge.next_node].out_degree != 1 or self.nodes[new_edge.next_node].in_degree != 1:
                                # del self.connections[new_edge.prev_node]
                                break
                    self.connections[node][new_edge_idx] = new_edge
        to_delete = set(self.nodes)
        for node in self.connections:
            self.nodes[node].out_degree = len(self.connections[node])
            for edge in self.connections[node]:
                to_delete.discard(edge.prev_node)
                to_delete.discard(edge.next_node)
        for node in list(to_delete):
            del self.nodes[node]

    def _find_tails(self, v):
        for edge in self.connections_temp[v]:
            if edge.coverage:
                edge.coverage -= 1
                if self.nodes[edge.next_node].out_degree <= 1 and edge.prev_node not in self.nodes_in_tail:
                    prev_start_node = edge.prev_node
                    start_node = edge.next_node
                    edge_temp = edge
                    tail_coverage = 0
                    tail_len = 0
                    while True:
                        if self.nodes[edge_temp.next_node].out_degree > 1:
                            break
                        tail_len += 1
                        tail_coverage += edge_temp.coverage
                        if self.nodes[edge_temp.next_node].out_degree == 0:
                            self.tails[(prev_start_node, start_node)] = tail_len*tail_coverage
                            break
                        edge_temp = self.connections[edge_temp.next_node][0]
                        self.nodes_in_tail.add(edge_temp.prev_node)
                self._find_tails(edge.next_node)

    def _check_tails(self):
        for node in self.nodes:
            if self.nodes[node].out_degree == 0:
                return True
        return False

    def delete_tails(self):
        if not self._check_tails():
            print('This graph does not have tails!')
            return
        # if self.deleted_tails:
        #     print('Tails have already been deleted')
        #     return
        # self.delete_tails = True
        self.nodes_in_tail = set()
        self.tails = {}
        in_degrees = [self.nodes[i].in_degree for i in self.connections.keys()]
        nodes_min_in_degrees = [list(self.connections.keys())[i] for i in range(len(in_degrees)) if in_degrees[i] == min(in_degrees)]
        self.connections_temp = deepcopy(self.connections)
        for start in nodes_min_in_degrees:
            self._find_tails(start)
        if len(self.tails) > 1:
            tails_min_scores = set(np.sort(list(self.tails.values()))[:int(len(self.tails)*0.3)+1])
            for tail in self.tails:
                if self.tails[tail] in tails_min_scores:
                    self.connections[tail[0]] = [edge for edge in self.connections[tail[0]] if edge.next_node != tail[1]]
                    self.nodes[tail[0]].out_degree = len(self.connections[tail[0]])
                    edge = self.connections[tail[1]][0]
                    while True:
                        del self.connections[edge.prev_node]
                        del self.nodes[edge.prev_node]
                        if not self.connections[edge.next_node]:
                            break
                        edge = self.connections_temp[edge.next_node][0]
                    del self.connections[edge.next_node]
                    del self.nodes[edge.next_node]

    def _print_connections(self):
        # count = 0
        # for node in self.nodes:
        #     if len(self.connections[node]) == 0:
        #         count += 1
        # print(count)
        print(self.connections)

    def _print_nodes(self):
        for node in self.nodes:
            print(node)
            print(self.nodes[node].in_degree)
            print(self.nodes[node].out_degree)

    def _find_euler_path(self, v):
        for edge in self.connections_temp[v]:
            if edge.coverage:
                edge.coverage -= 1
                self.string += edge.string[(self.k-1):]
                self._find_euler_path(edge.next_node)

    def assemble(self, multiple_start=False):
        in_degrees = [self.nodes[i].in_degree for i in self.connections.keys()]
        nodes_min_in_degrees = [list(self.connections.keys())[i] for i in range(len(in_degrees)) if in_degrees[i] == min(in_degrees)]
        if multiple_start:
            for start in nodes_min_in_degrees:
                self.connections_temp = deepcopy(self.connections)
                self.string = self.connections[start][0].string[:(self.k-1)]
                self._find_euler_path(start)
        else:
            start = nodes_min_in_degrees[0]
            self.connections_temp = deepcopy(self.connections)
            self.string = self.connections[start][0].string[:(self.k-1)]
            self._find_euler_path(start)
        return self.string
    
    def visualize(self, interactive=False, show_edges=False, layout=nx.kamada_kawai_layout):
        G = nx.DiGraph() 
        for node in self.connections:
            for edge in self.connections[node]:
                G.add_edge(edge.prev_node, edge.next_node, string=edge.string)
        if interactive:
            nt = Network('2000px', '2000px', directed=True)
            nt.from_nx(G)
            nt.show('test.html', notebook=False)
        else:
            for node in self.nodes:
                G.add_node(node)
            pos = layout(G)
            plt.figure(figsize=(15,15))
            nx.draw_networkx(G, pos, font_size=5)
            if show_edges:
                nx.draw_networkx_edge_labels(G, pos, font_size=8)
            plt.show()

    def get_statistics(self):
        edge_coverage = []
        node_degrees = []
        n_edges = 0
        n_nodes = len(self.nodes)
        for node in self.connections:
            node_degrees.append(self.nodes[node].in_degree + self.nodes[node].out_degree)
            for edge in self.connections[node]:
                edge_coverage.append(edge.coverage)
                n_edges += 1
        print('Number edges:', n_edges)
        print('Number nodes:', n_nodes)
        figure, axis = plt.subplots(1, 2, figsize=(25, 5))
        counts_edge, bins_edge = np.histogram(edge_coverage)
        axis[0].hist(bins_edge[:-1], 10, weights=counts_edge)
        axis[0].set_title('Edge coverage')
        # axis[0].set_xticks(np.arange(1, max(edge_coverage)+1, len(edge_coverage)//10))
        counts_node, bins_node = np.histogram(node_degrees)
        axis[1].hist(bins_node[:-1], 10, weights=counts_node)
        axis[1].set_title('Node degrees')
        # axis[1].set_xticks(np.arange(1, max(node_degrees)+1, len(node_degrees)//10))
        plt.show()
    
    def get_shared_motif(self, true_seq):
        if not self.string:
            print('Do assemble first!')
            return
        s_seq = self.string if len(self.string) <= len(true_seq) else true_seq
        l_seq = self.string if len(self.string) > len(true_seq) else true_seq
        motif = set()
        for i in range(len(s_seq)):
            for j in range(i+1, len(s_seq)+1):
                motif.add(s_seq[i:j])
        update_motif = list(motif)
        for m in update_motif:
            if m not in l_seq:
                motif.remove(m)
        motif = list(motif)
        longest_motif = motif[np.argmax([len(i) for i in motif])]
        print('Longest shared motif is:\n', longest_motif, end='\n\n\n')

        

true_seq_1 = 'AGATGAATGGACCGGCCATATAAGTAAG'
test_sequences = ['CCGTTG', 'CGTTAC', 'GTTGCA', 'TGCAGG']


G = DBGraph([true_seq], 6)
G.gen_kmers()
G.create_graph()
s1 = true_seq
s2 = G.assemble()
# G._print_connections()
# G._print_connections()

# G.visualize(show_edges=False, layout=nx.kamada_kawai_layout)
# G.delete_tails()
# print(s1)
# print(s2)
# G.get_shared_motif(s1)
# print()
# G.visualize(show_edges=False, layout=nx.kamada_kawai_layout)
# G.get_statistics()
# G.visualize(show_edges=False, layout=nx.kamada_kawai_layout)

# G.compact()
# G._print_nodes()
# G.visualize(show_edges=False, layout=nx.kamada_kawai_layout)
# G.get_shared_motif(s1)
# G.get_statistics()
# G._print_connections()

# G.delete_tails()
# G.compact()
G.visualize(interactive=True, show_edges=False, layout=nx.kamada_kawai_layout)
# G._print_nodes()


# G.compact()


# # s1 = true_seq
# # s2 = G.assemble()
# print(s1)
# print(s2)
# # print()
# G.visualize(show_edges=False, layout=nx.kamada_kawai_layout)
# G._print_connections()
# G.get_statistics()
# G.get_shared_motif(s1)

test.html


## Сжатие графа  
Научитесь производить сжатие графа Де Брюина. При сжатии не забывайте склеивать подстроки на ребрах и обновлять покрытие ребер. Пересчитывайте покрытие склеиваемых ребер как взвешенное среднее, где вес соответствует длине подстроки, соответствующей ребру.